In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
### set working directory and add to path
import sys
import os

path="/content/drive/MyDrive/cod/RL_DS/pyworld2-main"
os.chdir(path)
sys.path.append(path+"/")

In [3]:
from world2_switch import run_full, run_analisis
from pyworld2 import world2

In [4]:
#P=[BRN, NRUN, FC, CIGN,POLN]

P=[0.04, 0.99, 0.99, 0.05, 0.99]  ##default scenario  ### el segundo se sale del limite
print(run_full(output_file=None))
print(run_full(P))


0.7969945682062864
0.7813220983425839


In [5]:
# prompt: how is the fuill name skopt to install

!pip install scikit-optimize


In [6]:
# prompt: use bayesiuan optimization and diferential evolution to optimize de run_full (function) defining values of P  and using this bounds: Lim = [[0.02,0.04],[0.1,1.0],[0.6,1.25],[0.02,0.04],[0.1,1.0]]

import numpy as np
from skopt import gp_minimize
from scipy.optimize import differential_evolution


# Define the objective function
def objective_function(params):
    P = list(params)
    #print("P", P) # Debug: print parameters
    try:
        res = run_full(P)*-1  # Assuming run_full accepts P as a parameter
        #print(f"res: {res}") #Debug
        # Replace with your actual objective function
        # This example minimizes the 'pop' value
        #objective = res['pop'][-1]
        objective = res
        return objective
    except Exception as e:
        print(f"An error occurred during optimization: {e}")
        # Handle the error appropriately, e.g., return a large value
        return 1e10  # Return a very large value to penalize failures


# Define the bounds for the parameters
Lim = [[0.02,0.04],[0.1,1.0],[0.6,1.25],[0.02,0.04],[0.1,1.0]]
#P=[BRN, NRUN, FC, CIGN, POLN]
## default:
### escenario base se sale CIGN o es CIDN
### escenario 4 se sale NRUN
### escenario 6 se sale CIGN




# Bayesian Optimization
print("Starting Bayesian Optimization")
res_gp = gp_minimize(objective_function, Lim, n_calls=80, random_state=0)
#print("Bayesian Optimization Result:", res_gp)
best_params_gp = res_gp.x
print("Best parameters (Bayesian):", best_params_gp)
#Run analysis with the best params
print(run_full(best_params_gp))

# # # Differential Evolution
# # print("Starting Differential Evolution")
# result = differential_evolution(objective_function, bounds=Lim, maxiter = 100, popsize=10)
# #print("Differential Evolution Result:", result)
# print("best_params_de:",result.x)
# print(run_full(  result.x))
# # print("Best Parameters (DE):", best_params_de)


Starting Bayesian Optimization


/usr/local/lib/python3.11/dist-packages/skopt/space/space.py:116: UserWarning: Dimension [0.02, 0.04] was inferred to Real(low=0.02, high=0.04, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, it will be inferred to Categorical(categories=(0.02, 0.04), prior=None). See the documentation of the check_dimension function for the upcoming API.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/skopt/space/space.py:116: UserWarning: Dimension [0.1, 1.0] was inferred to Real(low=0.1, high=1.0, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, it will be inferred to Categorical(categories=(0.1, 1.0), prior=None). See the documentation of the check_dimension function for the upcoming API.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/skopt/space/space.py:116: UserWarning: Dimension [0.6, 1.25] was inferred to Real(low=0.6, high=1.25, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, i

Best parameters (Bayesian): [0.02, 0.1, 1.25, 0.02, 1.0]
3.96367836534156


In [ ]:
#Best Parameters (DE): [0.02, 0.1, 0.6, 0.04, 1.0] reward: 1.4340343977566994
#Best parameters (Bayesian): [0.02, 0.1, 0.6, 0.04, 1.0]  reward: 1.4340343977566994

In [7]:
P_DE=[0.02, 0.1, 0.6, 0.04, 1.0]
P_BO=[0.02, 0.1, 1.25, 0.02, 1.0]

print(run_full(P_DE))
print(run_full(P_BO))

1.2895247980004525
3.96367836534156
